If Dockerfiles have not been modified, connect to the Jupyter server with ```http://localhost:8005/tree?token=generate-segmentation```  

This notebook describes a pipeline to segment an image into its ground (v000), sky (vsky), vertical (v090), and porous (hpor).  
It uses [ImageMagick](https://imagemagick.org/), [graph-segmentation](https://github.com/fsrajer/graph-segmentation), an implementation of Felzenszwalb and Huttenlocher (2004), and the [Photo Pop-up software](https://dhoiem.cs.illinois.edu/projects/software.html) used in Hoiem, Efros, and Hebert (2005).

> Felzenszwalb, P.F., Huttenlocher, D.P. Efficient Graph-Based Image Segmentation. International Journal of Computer Vision 59, 167–181 (2004). https://doi.org/10.1023/B:VISI.0000022288.19776.77  

> D. Hoiem, A. A. Efros and M. Hebert, "Geometric context from a single image," Tenth IEEE International Conference on Computer Vision (ICCV'05) Volume 1, Beijing, China, 2005, pp. 654-661 Vol. 1, doi: 10.1109/ICCV.2005.107  

In [ ]:
target_dir = "data"
input_dir = "place-pulse-singapore-panos"
output_dir = "place-pulse-singapore-segmentations"

In [2]:
import os
from pathlib import Path

In [3]:
# %env LD_LIBRARY_PATH=/usr/local/lib:/app/v73/runtime/glnx86:/app/v73/sys/os/glnx86:/app/v73/sys/java/jre/glnx86/jre1.5.0/lib/i386/native_threads:/app/v73/sys/java/jre/glnx86/jre1.5.0/lib/i386/client:/app/v73/sys/java/jre/glnx86/jre1.5.0/lib/i386:/app/v73/bin/glnx86
# %env XAPPLRESDIR=/app/v73/X11/app-defaults

In [4]:
input_path = os.path.join(target_dir, input_dir)
output_path = os.path.join(target_dir, output_dir)
Path(output_path).mkdir(parents=True, exist_ok=True)
convert_dir = os.path.join(os.getcwd(), "convert")
Path(convert_dir).mkdir(parents=True, exist_ok=True)
segment_dir = os.path.join(os.getcwd(), "graph-segmentation")
segment_in_dir = os.path.join(segment_dir, "input")
segment_out_dir = os.path.join(segment_dir, "output")
photopopup_dir = os.path.join(os.getcwd(), "app")
photopopup_in_dir = os.path.join(photopopup_dir, "input")
photopopup_out_dir = os.path.join(photopopup_dir, "output")


In [5]:
!cp -a {input_path}/. {convert_dir}/
!magick mogrify -format ppm {convert_dir}/*.jpg

In [6]:
!cp {convert_dir}/*.ppm {segment_in_dir}
segment_shell_script = f"""count=0
for ppm in {segment_in_dir}/*.ppm; do
    filename=$(basename $ppm)
    filename=${{filename%.*}}
    {segment_dir}/segment 0.8 100 100 "$ppm" {segment_out_dir}/"$filename".pnm
    count=$((count+1))
    echo $count
done
"""
with open("segment.sh", 'w') as fp:
    fp.write(segment_shell_script)
!sh segment.sh

loading input image.
processing
got 442 components
done! uff...thats hard work.
1
loading input image.
processing
got 775 components
done! uff...thats hard work.
2
loading input image.
processing
got 487 components
done! uff...thats hard work.
3
loading input image.
processing
got 872 components
done! uff...thats hard work.
4
loading input image.
processing
got 581 components
done! uff...thats hard work.
5
loading input image.
processing
got 700 components
done! uff...thats hard work.
6
loading input image.
processing
got 487 components
done! uff...thats hard work.
7
loading input image.
processing
got 509 components
done! uff...thats hard work.
8
loading input image.
processing
got 427 components
done! uff...thats hard work.
9
loading input image.
processing
got 696 components
done! uff...thats hard work.
10
loading input image.
processing
got 544 components
done! uff...thats hard work.
11
loading input image.
processing
got 490 components
done! uff...thats hard work.
12
loading input

In [7]:
!cp -a {input_path}/. {photopopup_in_dir}
!cp -a {segment_out_dir}/*.pnm {photopopup_in_dir}
photopopup_shell_script = f"""count=0
cd {photopopup_dir}
for jpg in input/*.jpg; do
    if [ ! -e "output/$(basename "$jpg" .jpg).v000.pgm" ]; then
        ./photoPopup ./classifiers_08_22_2005.mat "$jpg" pnm ./output
    fi
    count=$((count+1))
    echo $count
done
"""
with open("photopopup.sh", 'w') as fp:
    fp.write(photopopup_shell_script)

Rerun the Photo Pop-up shell script below if the software appears to have frozen.  
It has a tendency to fail when writing the ```.wrl``` file and freeze.

In [ ]:
!sh photopopup.sh

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


If you had to manually stop and rerun the code above, manually run the code below to transfer the desired segmentated images out.

In [ ]:
!cp -a {photopopup_out_dir}/*.hpor.pgm {output_path}
!cp -a {photopopup_out_dir}/*.v000.pgm {output_path}
!cp -a {photopopup_out_dir}/*.v090.pgm {output_path}
!cp -a {photopopup_out_dir}/*.vsky.pgm {output_path}

cp: cannot stat '/app/app/output/*.hpor.pgm': No such file or directory
cp: cannot stat '/app/app/output/*.v000.pgm': No such file or directory
cp: cannot stat '/app/app/output/*.v090.pgm': No such file or directory
cp: cannot stat '/app/app/output/*.vsky.pgm': No such file or directory
